Are certain beer styles more popular per borough?

In [2]:
boroughs = ['Manhattan', 'Bronx', 'Brooklyn', 'Queens', 'Staten Island']
styles = {
    'Pilsner':['Pils', 'Light', 'American Lager', 'Pale Lager'],
    'Wheat':['Hefewiezen', 'Hefeweizen', 'Wiess', 'Wit', 'Witbier'],
    'Lager':['Vienna',
    'Stout':['Porter', 'Scotch', 'Smoked',
    'Ale': ['Pale Ale', 'Amber', 'Brown', 'Red', 'Saison', 'Bock', 'Dopplebock', 'Kolsch', 'Octoberfest', 'Cream' ],
    'IPA':['Session', 'Double IPA', 'IIPA', '2xIPA', '2XIPA', 'NEIPA']
    'Sour':['Gose', 'Berliner Weisse', 
    'Imperial Stout':['Barley Wine', 
    'Belgian Strong':['Tripel', 'Dubbel', 'Abbey', 
    'Fruit':['Lambic', 'Pumpkin', 'Cider']
    }